# Parameter sensitivity in the SKA Performance Model

This notebook is originally work by Tim Cornwell, providing the plots for the TCC memo TCC-SDP-151123-1-1.

In [ ]:
import sys

from ipywidgets import interact, interact_manual, fixed
from IPython.display import display, HTML
import matplotlib.pyplot as plt

sys.path+=['..']
from sdp_par_model import evaluate
from sdp_par_model import reports as iapi
from sdp_par_model.config import PipelineConfig
from sdp_par_model.parameters.definitions import *

# Do plotting inline since sending to a specific file in jupyter/matplotlib is fiddly
%matplotlib inline

verbose = False

Set up routine for doing parameter sweeps

In [ ]:
def make_sweep(parameter, param_val_min, param_val_max, number_steps,
               telescope, band, immodes, expressions, stack=False):
    for immode in immodes:
        resultss = []
        cfg = PipelineConfig(telescope=telescope, pipeline=immode, band=band)
        (param_values, resultss) = cfg.eval_param_sweep_1d(expressions, parameter_string=parameter, param_val_min=param_val_min,
                                                           param_val_max=param_val_max, number_steps=number_steps, 
                                                           verbose=verbose)
        if stack:
            plt.figure(figsize=(8,6))
            plt.title('%s sweep for %s in %s mode ' % (parameter, telescope, immode))
            plt.stackplot(param_values, *resultss, labels=expressions)
            handles, labels = plt.gca().get_legend_handles_labels()
            plt.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
            plt.show()
        else:
            for expression, results in zip(expressions, resultss):
                header = '%s for %s in %s mode as a function of %s' % (expression, telescope, immode, parameter)
                iapi.plot_line_datapoints(header, param_values, np.array(results), xlabel=parameter, ylabel=expression)

Determine main cost drivers to show in stacked view

In [ ]:
tp = PipelineConfig(telescope=Telescopes.SKA1_Low, pipeline=Pipelines.ICAL, band=Bands.Low, Nfacet=5).calc_tel_params()
main_products = list(sorted([ p for p in tp.products if tp.products[p]['Rflop'] > 1e9 ]))
main_product_flops = list([ "{}.Rflop".format(prod) for prod in main_products ])

## Evaluate scaling as a function of Q_FOV

In [ ]:
make_sweep('Qfov', 1.0, 3.0, 10, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], main_product_flops, stack=True)
make_sweep('Qfov', 1.0, 3.0, 10, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], 
           [ 'Nfacet', 'Rio', 'Rfacet_vis', 'Rimage'])
make_sweep('Qfov', 1.0, 3.0, 10, Telescopes.SKA1_Mid, Bands.Mid1, [Pipelines.ICAL], main_product_flops, stack=True)
make_sweep('Qfov', 1.0, 3.0, 10, Telescopes.SKA1_Mid, Bands.Mid1, [Pipelines.ICAL],
           [ 'Nfacet', 'Rio', 'Rfacet_vis', 'Rimage'])

## Evaluate scaling as a function of Bmax

In [ ]:
make_sweep('Bmax', 10000, 65000, 9, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], main_product_flops, stack=True)
make_sweep('Bmax', 10000, 65000, 9, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], [ 'Nfacet', 'Rio', 'Rfacet_vis'])
make_sweep('Bmax', 15000, 150000, 9,Telescopes.SKA1_Mid, Bands.Mid1, [Pipelines.ICAL], main_product_flops, stack=True)
make_sweep('Bmax', 15000, 150000, 9,Telescopes.SKA1_Mid, Bands.Mid1, [Pipelines.ICAL], [ 'Nfacet', 'Rio', 'Rfacet_vis'])

## Evaluate scaling as a function of Tion

In [ ]:
make_sweep('Tion', 1, 10, 9, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], main_product_flops, stack=True)
make_sweep('Tion', 1, 10, 9, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], [ 'Rflop', 'Rfacet_vis'])

## Evaluate scaling as a function of A-kernel size

In [ ]:
make_sweep('Naa_a', 5,11, 9, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], [ 'Rflop', 'Nfacet' ])
make_sweep('Naa_a', 5,11, 9, Telescopes.SKA1_Mid, Bands.Mid1, [Pipelines.ICAL], [ 'Rflop', 'Nfacet' ])

## Evaluate scaling of image-domain gridding

In [ ]:
make_sweep('image_gridding', 12,24, 6, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], main_product_flops, stack=True)
make_sweep('image_gridding', -1,0, 1, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL], main_product_flops, stack=True)

In [ ]:
make_sweep('image_gridding', 12,24, (24-12)//2, Telescopes.SKA1_Low, Bands.Low, [Pipelines.ICAL],
           [ 'Nslack_pred', 'Nwstack' ])